In [4]:
from utils import *
check_tensorflow_gpu()
K.clear_session()

proxy = SpatiotemporalCO2()

proxy.num_epochs = 30
proxy.batch_size = 20

Checking Tensorflow Version:
Tensorflow built with CUDA? True
Tensorflow version: 2.10.0
# GPU available: 1
CUDA: 64_112 | cuDNN: 64_8
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
proxy.load_data()
proxy.process_data(subsample=300)

X: (1000, 64, 64, 4) | y: (1000, 60, 64, 64, 2)
normalized - X: (1000, 64, 64, 4) | y: (1000, 60, 64, 64, 2)
Subsampling data for 100 samples ...
Train - X: (75, 64, 64, 4) | y: (75, 60, 64, 64, 2)
Test  - X: (25, 64, 64, 4) | y: (25, 60, 64, 64, 2)


In [6]:
model0 = proxy.make_model()
model, fit = proxy.train(model0)

# Parameters: 458,514


ResourceExhaustedError: Graph execution error:

Detected at node 'model/conv_lstm2d_2/transpose' defined at (most recent call last):
    File "c:\Users\Misael Morales\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\Misael Morales\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\Misael Morales\AppData\Roaming\Python\Python39\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\Misael Morales\AppData\Local\Programs\Python\Python39\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Users\Misael Morales\AppData\Roaming\Python\Python39\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\Misael Morales\AppData\Roaming\Python\Python39\site-packa

In [ ]:
y_train_pred = model.predict(proxy.X_train)
y_test_pred  = model.predict(proxy.X_test)
print('Train pred: {} | Test pred: {}'.format(y_train_pred.shape, y_test_pred.shape))

In [ ]:
k = 78
c = 1

fig, axs = plt.subplots(2, 12, figsize=(20,4))
for j in range(12):
    axs[0,j].imshow(proxy.y_train[k,j*5,:,:,c], 'jet')
    axs[1,j].imshow(y_train_pred[k,j*5,:,:,c], 'jet')
    axs[0,j].set(title='t={}'.format(j*5))
    for i in range(2):
        axs[i,j].set(xticks=[], yticks=[])
axs[0,0].set(ylabel='True'); axs[1,0].set(ylabel='Pred')
plt.show()

***
# END